# Combine trait-mapping data with peptide predictions and summarize

This notebook creates a summary TSV file that records incidence of peptide predictions within each orthogroup that is associated with the itch suppression trait.
This information should help us select orthogroups that have:
1. Evidence of peptide predictions (peptide predictions many proteins in the orthogroup)
2. Evidence of export to the saliva (contains a signal peptide, has a tick salivary gland transcriptome hit)
3. Support for itch suppression. The strongest signal would be protein families from tick + non-tick that suppress itch. The trait mapping coefficients should also help identify this.

Some thoughts from Austin are included below.
From Austin in slack (edited for clarity):
> As for mixed orthogroups. Yes, I would absolutely expect these orthogroups to possess proteins/peptides present in both itch-suppressing and non-itch-suppressing species.
> * These tests are not looking for pure presence/absence patterns (i.e. completely absent in non-suppressing and present in suppressing species) - they are looking for gross patterns.
> * At the level of profile clusters, I took the mean counts of different event types, e.g., speciation events in the gene family (any bifurcation in the gene family tree) within itch-suppressing species and non-itch-suppressing species.
> * I then conducted logistic regressions, asking whether the mean event counts predicted itch suppression (binary, 0/1).
>    * So long as the counts significantly differed between itch-suppressing and non-itch-suppressing species (and the coefficients are sufficiently large to be retained, depending on how you filtered), they will be included in your set of “itch-suppression-associated” clusters
> * Within the profile cluster, I then conducted logistic regressions for each gene family individual, this time regressing the event counts for each species against the response - the binary itch suppression trait.
> * One key bit is that this time, I used a phylogenetic logistic regression that accounts for the evolutionary non-independence.
>    * That will ultimately mean that in a hypothetical scenario where all ticks have one or more gene copies of something, but no other species do (including other non-tick itch-suppressing species), this is unlikely (or less likely) to lead to a significant association. This is because although we have many ticks in the dataset, they're effectively evolutionary pseudoreplicates, as that gene is only associated with a single evolutionary “origin” or incidence of itch suppression.
>    * The association between that gene and itch suppression is confounded with the association between that gene and… ticks. And all other traits are unique to them, independent of itch suppression. 
> * These are just statistical associations between gene family event counts in profile clusters or gene families within them and itch suppression.
>    * Because they are associations between these different event types and not just gene presence/absence, they will include cases where the gene is typically absent in one group and present in the other, as well as cases where it’s present in both, but the “magnitude” or frequency of event counts differs.
>
> As a final thought: I think one thing that would be particularly useful to consider in your filtering is to consider not only those gene families where peptides are present in multiple ticks but multiple ticks as well as other non-tick itch-suppressing species. The statistical tests should already be “prioritizing” these.

In [1]:
library(tidyverse)
library(UpSetR)

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
setwd("..")

In [3]:
# adjust plot size rendered inline
options(repr.plot.width = 6, repr.plot.height = 4, repr.plot.res = 300)

## Define some descriptive variables associated with trait mapping

In [4]:
evidence_of_itch_suppression_species <- c("Sarcoptes scabiei",
                                          "Psoroptes ovis",
                                          "Amblyomma americanum",
                                          "Amblyomma sculptum",
                                          "Dermacentor andersoni",
                                          "Dermacentor silvarum",
                                          "Dermacentor variabilis",
                                          "Haemaphysalis longicornis",
                                          "Hyalomma asiaticum",
                                          "Ixodes persulcatus",
                                          "Ixodes ricinus",
                                          "Ixodes scapularis",
                                          "Rhipicephalus microplus",
                                          "Rhipicephalus sanguineus")

In [5]:
ticks <- c("Amblyomma americanum",
           "Amblyomma sculptum",
           "Dermacentor andersoni",
           "Dermacentor silvarum",
           "Dermacentor variabilis",
           "Haemaphysalis longicornis",
           "Hyalomma asiaticum",
           "Ixodes persulcatus",
           "Ixodes ricinus",
           "Ixodes scapularis",
           "Rhipicephalus microplus",
           "Rhipicephalus sanguineus")

## Read in initial data

Note that for the peptigate results, we remove propeptide predictions -- DeepPeptide predict peptides and propeptides.
The tool uses the [UniProt definition of propeptide](https://www.uniprot.org/help/propep).
> A **Propeptide** is a part of a protein that is cleaved during maturation or activation of the protein. It is generally understood not to have an independent function.
A **Peptide** is proteolytically cleaved and has a well-defined biological activity.

Given this, we remove propeptide predictions as DeepPeptide would predict that these are not biologically active after being cleaved from their precursor protein.

In [6]:
# tot peptigate predictions & remove propeptide predictions
peptigate_predictions <- read_tsv("outputs/ToT_20240626/predictions/peptide_predictions.tsv", show_col_types = F) %>%
  rename_with(.cols = everything(), function(x){paste0("peptigate_", x)}) %>%
  mutate(peptigate_peptide_class = ifelse(is.na(peptigate_peptide_class), "sORF", peptigate_peptide_class)) %>%
  filter(peptigate_peptide_class != "Propeptide") %>%
  mutate(traitmapping_locus_tag = gsub("_start.*", "", peptigate_peptide_id))

In [7]:
# note that some proteins have multiple peptide predictions, so we have to be careful how we do our counting later in this notebook.
nrow(peptigate_predictions)
length(unique(peptigate_predictions$traitmapping_locus_tag))

[1] 308

[1] 283

In [8]:
# transcriptome shotgun assembly (TSA) salivary gland (sg) transcriptome peptide prediction BLAST hits
tsa_sg_blastp <- read_tsv("outputs/analysis/compare_tsa_sg/tsa_sg_peptides_blastp_matches.tsv", show_col_types = FALSE) %>%
  # select only one blast hit for each query peptide
  group_by(qseqid) %>%
  slice_max(bitscore) %>%
  slice_min(evalue) %>%
  slice_head(n = 1) %>%
  ungroup() %>%
  filter(qseqid %in% peptigate_predictions$peptigate_peptide_id) %>% # filter out hits to propeptides
  rename_with(.cols = everything(), function(x){paste0("sgpeptide_blast_", x)})

In [9]:
# double check that each qseqid only appears once
nrow(tsa_sg_blastp)
length(unique(tsa_sg_blastp$sgpeptide_blast_qseqid))

[1] 146

[1] 146

In [10]:
# metadata from trait mapping ToT data for association with itch-suppression
trait_mapping_metadata <- read_tsv("inputs/2024-06-26-top-positive-significant-clusters-orthogroups-annotations.tsv.gz", show_col_types = F) %>%
  mutate(species = gsub("-", " ", species)) %>%
  rename_with(.cols = everything(), function(x){paste0("traitmapping_", x)})

## Use the trait mapping metadata as well as peptigate results and analysis of those results to understand the breakdown of the different orthogroups

* number of proteins in orthogroup
* number of proteins in orthogroup with evidence of expression in salivary gland transcriptomes from the TSA (and percent)
* number of proteins in orthogroup with peptide prediction (and percent)
* number of proteins in orthogroup from ticks (order Ixodida) (and percent)
* number of proteins in orthogroup from chelicerates that suppress itch (and percent)

### Count traits of PROTEINS (not peptides from ticks) in the orthogroup.

Includes:
- how many proteins were in the orthogroup
- how many proteins in the orthogroup were from ticks
- how many proteins in the orthogroup had a signal peptide
- how many protein in orthogroup are from species that suppress itch (note itchsuppsp stands for "itch suppression species")

In [11]:
# Calculate some of the statistics prior to joining with other metadata.
# Some proteins had multiple peptides predicted from them, so we should calculate some numbers prior to joining data
num_proteins_in_orthogroup <- trait_mapping_metadata %>%
  group_by(traitmapping_orthogroup) %>%
  summarize(num_proteins_in_orthogroup = n())

In [12]:
traits_of_proteins_in_orthogroup <- trait_mapping_metadata %>%
  mutate(species_is_a_tick = ifelse(traitmapping_species %in% ticks, "tick", "not tick"),
         species_suppresses_itch = ifelse(traitmapping_species %in% evidence_of_itch_suppression_species, "evidence suppresses itch", "no evidence suppresses itch")) %>%
  left_join(num_proteins_in_orthogroup, by = "traitmapping_orthogroup") %>%
  group_by(traitmapping_orthogroup, num_proteins_in_orthogroup) %>%
  # number of proteins in orthogroup from ticks (order Ixodida) (and fraction), and how many had peptide predictions
  summarize(num_tick_proteins_in_orthogroup = sum(species_is_a_tick == "tick"),
            fraction_of_orthogroup_tick_proteins = num_tick_proteins_in_orthogroup / num_proteins_in_orthogroup,
            num_tick_proteins_in_orthogroup_with_signal_peptide = sum(traitmapping_deepsig_feature == "Signal peptide", na.rm = TRUE),
            fraction_of_orthogroup_with_signal_peptide = num_tick_proteins_in_orthogroup_with_signal_peptide / num_proteins_in_orthogroup,
            num_itchsuppsp_proteins_in_orthogroup = sum(species_suppresses_itch == "evidence suppresses itch"),
            fraction_of_orthogroup_itchsuppsp_proteins = num_itchsuppsp_proteins_in_orthogroup / num_proteins_in_orthogroup) %>%
  distinct()

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()` always returns an ungrouped data frame and adjust accordingly.”
`summarise()` has grouped output by 'traitmapping_orthogroup', 'num_proteins_in_orthogroup'. You can override using the `.groups` argument.


### Calculate things for peptides

Includes
- number of proteins with a predicted peptide. Some proteins had multiple peptides. For this metric, we only count those things once.
- number of predicted sORF peptides per orthogroup
- number of predicted sORF peptides with a signal peptide
- number of predicted cleavage peptides per orthogroup
- number of proteins the cleavage peptides are from

In [13]:
# Some proteins had multiple peptides predicted from them.
# Only count the protein once, even if it had multiple peptides predicted from it.
num_proteins_with_predicted_peptide <- trait_mapping_metadata %>%
  left_join(peptigate_predictions, by = "traitmapping_locus_tag") %>% 
  mutate(peptigate_peptide_prediction = ifelse(is.na(peptigate_peptide_class), "no peptide prediction", "peptide prediction")) %>%
  # Arbitrarily select one peptide prediction for each protein, just for counting.
  # This shouldn't impact selection of whether there is a peptide prediction, since only peptides with multiple predictions will be removed
  group_by(traitmapping_locus_tag, traitmapping_orthogroup) %>%
  slice_head(n = 1) %>%
  ungroup() %>%
  left_join(num_proteins_in_orthogroup, by = "traitmapping_orthogroup") %>%
  group_by(traitmapping_orthogroup, num_proteins_in_orthogroup) %>%
  summarize(num_proteins_with_a_predicted_peptide = sum(peptigate_peptide_prediction == "peptide prediction"),
            fraction_of_orthogroup_with_predicted_peptide = num_proteins_with_a_predicted_peptide / num_proteins_in_orthogroup) %>%
  distinct() %>%
  arrange(desc(fraction_of_orthogroup_with_predicted_peptide))

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()` always returns an ungrouped data frame and adjust accordingly.”
`summarise()` has grouped output by 'traitmapping_orthogroup', 'num_proteins_in_orthogroup'. You can override using the `.groups` argument.


In [14]:
# average number of peptide predictions per protein within orthogroup
mean_num_peptides_predicted_per_protein_in_orthogroup <- trait_mapping_metadata %>%
  left_join(peptigate_predictions, by = "traitmapping_locus_tag") %>% 
  group_by(traitmapping_orthogroup, traitmapping_locus_tag) %>%
  tally() %>% 
  group_by(traitmapping_orthogroup) %>%
  summarize(mean_num_peptides_predicted_per_protein_in_orthogroup = mean(n)) %>%
  arrange(desc(mean_num_peptides_predicted_per_protein_in_orthogroup))

In [15]:
traits_of_peptides_in_orthogroup <- trait_mapping_metadata %>%
  # join to peptide predictions
  # note this will produce double counting for any locus_tag (protein) with multiple peptide predictions
  left_join(peptigate_predictions, by = "traitmapping_locus_tag") %>%
  # join to peptide prediction hits in the tick salivary gland transcriptomes
  left_join(tsa_sg_blastp, by = c("peptigate_peptide_id" = "sgpeptide_blast_qseqid")) %>%
  mutate(peptigate_peptide_prediction = ifelse(is.na(peptigate_peptide_class), "no peptide prediction", "peptide prediction"),
         species_suppresses_itch = ifelse(traitmapping_species %in% evidence_of_itch_suppression_species, "evidence suppresses itch", "no evidence suppresses itch"),
         species_is_a_tick = ifelse(traitmapping_species %in% ticks, "tick", "not tick")) %>%
  group_by(traitmapping_cluster, traitmapping_orthogroup, traitmapping_coefficient, traitmapping_signif_level, traitmapping_signif_fdr) %>%
  summarize(num_predicted_peptides = sum(peptigate_peptide_prediction == "peptide prediction"),
            num_predicted_peptides_from_tick = sum(peptigate_peptide_prediction == "peptide prediction" & species_is_a_tick == "tick"),
            num_predicted_peptides_from_itchsuppsp = sum(peptigate_peptide_prediction == "peptide prediction" & species_suppresses_itch == "evidence suppresses itch"), 
            num_predicted_peptides_with_sg_blast_hit = sum(!is.na(sgpeptide_blast_bitscore)),
            num_predicted_peptides_with_signal_peptide = sum(traitmapping_deepsig_feature == "Signal peptide" & peptigate_peptide_prediction == "peptide prediction"),
            num_sORF_peptide_predictions = sum(peptigate_peptide_type == "sORF", na.rm = TRUE),
            num_sORF_with_signal_peptide = sum(peptigate_peptide_type == "sORF" & traitmapping_deepsig_feature == "Signal peptide", na.rm = TRUE),
            num_cleavage_peptide_predictions = sum(peptigate_peptide_type == "cleavage", na.rm = TRUE),
            num_cleavage_with_signal_peptide = sum(peptigate_peptide_type == "cleavage" & traitmapping_deepsig_feature == "Signal peptide", na.rm = TRUE),
            type_of_itch_suppression_evidence = ifelse(num_predicted_peptides_from_itchsuppsp > num_predicted_peptides_from_tick, "chelicerate support", 
                                                       ifelse(num_predicted_peptides_from_tick > 0, "tick support", "no support"))
  ) %>%
  ungroup() %>%
  arrange(desc(num_predicted_peptides_with_signal_peptide))

`summarise()` has grouped output by 'traitmapping_cluster', 'traitmapping_orthogroup', 'traitmapping_coefficient', 'traitmapping_signif_level'. You can
override using the `.groups` argument.


In [18]:
orthogroup_peptide_summary <- left_join(num_proteins_with_predicted_peptide, traits_of_proteins_in_orthogroup, by = c("traitmapping_orthogroup", "num_proteins_in_orthogroup")) %>%
  left_join(mean_num_peptides_predicted_per_protein_in_orthogroup, by = "traitmapping_orthogroup") %>%
  left_join(traits_of_peptides_in_orthogroup, by = "traitmapping_orthogroup")

In [19]:
orthogroup_peptide_summary_filtered <- orthogroup_peptide_summary %>%
  # filter out orthogroups that had no predicted peptides
  filter(num_predicted_peptides > 0) %>%
  # filter out orthogroups that had no evidence of being expressed in salivary gland
  filter(num_predicted_peptides_with_sg_blast_hit > 0) %>%
  arrange(desc(traitmapping_coefficient))

orthogroup_peptide_summary_filtered 

traitmapping_orthogroup,num_proteins_in_orthogroup,num_proteins_with_a_predicted_peptide,fraction_of_orthogroup_with_predicted_peptide,num_tick_proteins_in_orthogroup,fraction_of_orthogroup_tick_proteins,num_tick_proteins_in_orthogroup_with_signal_peptide,fraction_of_orthogroup_with_signal_peptide,num_itchsuppsp_proteins_in_orthogroup,fraction_of_orthogroup_itchsuppsp_proteins,⋯,num_predicted_peptides,num_predicted_peptides_from_tick,num_predicted_peptides_from_itchsuppsp,num_predicted_peptides_with_sg_blast_hit,num_predicted_peptides_with_signal_peptide,num_sORF_peptide_predictions,num_sORF_with_signal_peptide,num_cleavage_peptide_predictions,num_cleavage_with_signal_peptide,type_of_itch_suppression_evidence
<chr>,<int>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
OG0011284,9,1,0.11111111,8,0.8888889,0,0.000000000,8,0.8888889,⋯,1,1,1,1,0,1,0,0,0,tick support
OG0008888,16,1,0.06250000,16,1.0000000,7,0.437500000,16,1.0000000,⋯,1,1,1,1,0,1,0,0,0,tick support
OG0001774,62,41,0.66129032,48,0.7741935,39,0.629032258,49,0.7903226,⋯,45,38,39,34,36,38,32,7,4,chelicerate support
OG0008102,20,14,0.70000000,20,1.0000000,14,0.700000000,20,1.0000000,⋯,18,18,18,13,13,3,2,15,11,tick support
OG0002194,56,2,0.03571429,45,0.8035714,1,0.017857143,45,0.8035714,⋯,2,2,2,1,0,2,0,0,0,tick support
OG0000189,240,21,0.08750000,193,0.8041667,22,0.091666667,195,0.8125000,⋯,21,20,20,15,0,20,0,1,0,tick support
OG0000746,102,5,0.04901961,71,0.6960784,2,0.019607843,72,0.7058824,⋯,5,4,4,2,0,5,0,0,0,tick support
OG0000194,237,23,0.09704641,203,0.8565401,16,0.067510549,203,0.8565401,⋯,23,21,21,10,0,23,0,0,0,tick support
OG0000880,93,68,0.73118280,92,0.9892473,53,0.569892473,92,0.9892473,⋯,84,82,82,30,40,0,0,84,40,tick support


In [21]:
# reorder the columns so the table is easier to interpret
orthogroup_peptide_summary_filtered <- orthogroup_peptide_summary_filtered %>%
  select(traitmapping_cluster, traitmapping_orthogroup, traitmapping_coefficient, traitmapping_signif_level, traitmapping_signif_fdr,
         num_proteins_in_orthogroup, num_proteins_with_a_predicted_peptide, fraction_of_orthogroup_with_predicted_peptide,
         num_predicted_peptides_with_signal_peptide, num_sORF_peptide_predictions, num_sORF_with_signal_peptide, 
         num_cleavage_peptide_predictions, num_cleavage_with_signal_peptide, num_predicted_peptides_with_sg_blast_hit,   
         num_predicted_peptides_from_tick, num_predicted_peptides_from_itchsuppsp, type_of_itch_suppression_evidence,  
         num_tick_proteins_in_orthogroup, fraction_of_orthogroup_tick_proteins,
         num_tick_proteins_in_orthogroup_with_signal_peptide, fraction_of_orthogroup_with_signal_peptide, 
         num_itchsuppsp_proteins_in_orthogroup, fraction_of_orthogroup_itchsuppsp_proteins) %>%
  arrange(desc(fraction_of_orthogroup_with_predicted_peptide))

In [22]:
write_tsv(orthogroup_peptide_summary_filtered, "outputs/notebooks/20240626_orthogroup_peptide_summary.tsv")

We made a data frame that summarizes our ToT peptigate results.
See a description of the columns below.

**Trait mapping results**: We included some relevant trait mapping analysis results to help contextualize the rest of the results.
- **traitmapping_cluster**: Identifier for the cluster within which the orthogroup was analyzed.
- **traitmapping_orthogroup**: Identifier for the orthogroup being analyzed.
- **traitmapping_coefficient**: Regression coefficient from the logistic regression associating orthogroup presence with itch suppression.
- **traitmapping_signif_level**: Indicates whether the association between the orthogroup and itch suppression is statistically significant.
- **traitmapping_signif_fdr**: False discovery rate adjusted p-value for the significance of the association.

**Relevant protein trait information**:
- **num_proteins_in_orthogroup**: Total number of proteins within the specified orthogroup.
- **num_proteins_with_a_predicted_peptide**: Count of proteins in the orthogroup with at least one predicted peptide.
- **fraction_of_orthogroup_with_predicted_peptide**: Fraction of proteins in the orthogroup with at least one predicted peptide.

**Peptide trait information**: Some proteins had multiple peptide predictions. These counts count all predicted peptides.
- **num_predicted_peptides_with_signal_peptide**: Number of predicted peptides that contain a signal peptide.
- **num_sORF_peptide_predictions**: Number of predicted peptides classified as sORF within the orthogroup.
- **num_sORF_with_signal_peptide**: Number of sORF peptides that contain a signal peptide.
- **num_cleavage_peptide_predictions**: Number of predicted peptides classified as cleavage peptides within the orthogroup.
- **num_cleavage_with_signal_peptide**: Number of cleavage peptides that contain a signal peptide.
- **num_predicted_peptides_with_sg_blast_hit**: Number of predicted peptides that matched sequences in the salivary gland transcriptomes.
- **num_predicted_peptides_from_tick**: Number of predicted peptides derived from tick species within the orthogroup.
- **num_predicted_peptides_from_itchsuppsp**: Number of predicted peptides derived from all itch-suppressing species within the orthogroup.
- **type_of_itch_suppression_evidence**: Either `chelicerate support`, `tick support`, or `no support`. `Chelicerate support` means that the orthogroup was associated with itch suppression and that there were peptides predicted from both ticks and non-tick itch suppressors that are chelicerates. We think these probably show some of the best evolutionary support for itch suppression but please still think critically about these predictions. `Tick support` means that the orthogroup was associated with itch suppression and peptides were only predicted from tick species. `No support` means no peptides were predicted from tick or the two non-tick itch suppressors.

**Other protein trait information**: Some other information about proteins may be useful, so we included these columns at the end of the data frame.
- **num_tick_proteins_in_orthogroup**: Number of proteins within the orthogroup that are derived from tick species.
- **fraction_of_orthogroup_tick_proteins**: Fraction of proteins in the orthogroup that are derived from tick species.
- **num_tick_proteins_in_orthogroup_with_signal_peptide**: Number of tick proteins within the orthogroup that have a signal peptide.
- **fraction_of_orthogroup_with_signal_peptide**: Fraction of proteins in the orthogroup with a signal peptide.
- **num_itchsuppsp_proteins_in_orthogroup**: Number of proteins within the orthogroup from species known to suppress itch.
- **fraction_of_orthogroup_itchsuppsp_proteins**: Fraction of proteins in the orthogroup from itch-suppressing species.


In [23]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/tidyjupyter/lib/libopenblasp-r0.3.26.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] UpSetR_1.4.0    lubridate_1.9.3 forcats_1.0.0   stringr_1.5.1  
 [5] dplyr_1.1.4     purrr_1.0.2     readr_2.1.5     tidyr_1.3.1    
 [9] tibble_3.2.1    ggplot2_3.5.0   tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] bit_4.0.5        gtable_0.3.4     jsonlite_1.8.8   compiler_4.3.3  
 [5] crayon_1.5.2     Rcpp_1.0.12      tidyselect_1.2.0 IRdisplay_1.1   
 [9] parallel_4.3.3   gridExtra_2.3    scales_1.3.0     uuid_1.2-0      
[13] fastmap_1.1.1    IRkernel_1.3.2  